In [19]:
pip install langchainhub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.16
    Uninstalling urllib3-1.26.16:
      Successfully uninstalled urllib3-1.26.16
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-auth 2.21.0 requires urllib3<2.0, but you have urllib3 2.2.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import tiktoken
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS


In [2]:
import langchain
print(f"LangChain Version: {langchain.__version__}")

LangChain Version: 0.1.7


In [15]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

In [16]:
loader = PyPDFLoader("/Users/jaehyeongpark/Documents/kb_report.pdf")
documents = loader.load_and_split()

In [17]:
documents[0].metadata

{'source': '/Users/jaehyeongpark/Documents/kb_report.pdf', 'page': 0}

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

In [19]:
len(documents)

17

In [20]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

In [21]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

In [22]:
retriever = vector_store.as_retriever()

In [23]:
retrieved_documents = retriever.invoke("주택 담보 대출의 동향은?")

In [24]:
for doc in retrieved_documents:
  print(doc)

page_content='2 \n \n목 차 \n \n \n \n \n \n· 주택 매매   …………………………………………………   4 \n· 주택 전세   …………………………………………………   5 \n· 주택 거래   …………………………………………………   6 \n· 주택 분양   …………………………………………………   7 \n· 토지        …………………………………………………   8 \n· 주택 금융   …………………………………………………   9 \n \n \n \n \n \n \n \n  주택 시장 동향' metadata={'source': '/Users/jaehyeongpark/Documents/kb_report.pdf', 'page': 1}
page_content='9 \n[주택 금융] 주택담보대출  증가세가 소폭 둔화 \n \n■ 2023년 12월 은행권  주택담보대출 (전세자금대출  포함)은 전월 대비 5.2조 원 증가 \n\uf06d 8월 전월 대비 7조 원 증가하며 최고치를  기록한  후 전반적으로  증가세 가 둔화되고  있으나 , 여전히  \n2022 년 월평균  순증액 (1.7조 원)을 크게 상회 \n∙ 전세자금대출  잔액은  2023 년 6월 이후 큰 변화 없이 보합세를  유지 \n\uf06d 거래 침체와  정부의  가계대출  관리 기조 등을 감안할  때 주택담보대출  증가세가  한풀 꺾일 것으로  전망 \n \n그림 20| 은행권  주택담보대출  규모 추이  그림 21| 은행권  전세자금대출  규모 추이 \n  \n \n주: 전세자금대출  등 주택 관련 대출 포함 \n자료: 한국은행   주: 월간 순증액은  한국은행  월간 수보 기준, 기금재원대출  제외 \n자료: 한국은행  \n \n■ 주택담보대출 (신규 취급액  기준) 금리는 2개월 연속 하락세를 이어감  \n\uf06d 2023 년 12월 신규 취급액  기준 주택담보대출  금리는  4.16% 로 전월 대비 0.32%p 하락 \n∙ 코픽스  금리(신잔액 )는 전월 대비 0.0

In [25]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [26]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


In [27]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [28]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

In [30]:
question = "주택 매매 동향은 어땧습니까?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

주택 매매가격은 하락세가 이어지고 있으며, 연초 이후 '하락 전망' 비중은 둔화되고 있습니다.


In [31]:
question = "주택 매매 동향은 어땧습니까?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

주택 매매가격은 하락세가 이어지고 있으며, 연초 이후 '하락 전망' 비중은 둔화되고 있습니다.
[Document(page_content='2 \n \n목 차 \n \n \n \n \n \n· 주택 매매   …………………………………………………   4 \n· 주택 전세   …………………………………………………   5 \n· 주택 거래   …………………………………………………   6 \n· 주택 분양   …………………………………………………   7 \n· 토지        …………………………………………………   8 \n· 주택 금융   …………………………………………………   9 \n \n \n \n \n \n \n \n  주택 시장 동향', metadata={'source': '/Users/jaehyeongpark/Documents/kb_report.pdf', 'page': 1}), Document(page_content='3 \n \n   \n     \n \n◦ 주택 매매가격은 하락세 가 이어지고  있으나 , 연초 이후 ‘하락 전망’ 비중은  둔화 \n∙ 1월 전국 주택 매매가격 은 전월 대비 0.12%  하락하며 전국 대부분  지역에서  하락세 를 보임 \n- 수도권과  비수도권에서  모두 하락세 를 이어감  \n- 5개광역시 는 하락세가  장기화 되고 있으며 , 대구에서  가장 큰 폭으로  하락 \n∙ 매매가격전망지수는  여전히  ‘하락 전망’ 비중이  크지만 , 연초 이후 ‘하락 전망’ 비중은  둔화 \n◦ 아파트  선호 현상으로  인해 수도권  아파트를  중심으로  전세가격  상승세 가 지속될 전망 \n∙ 주택 전세가격 은 전월 대비 0.09% 상승하며  5개월 연속 상승세 나, 시장 분위기는  지역별로  차이를 보임 \n- 수도권의  경우 서울, 경기 지역은  상승세를  보이는  반면 인천은  2개월 연속 하락세  \n- 비수도권은  대전을  제외한  전국 대부분  지역에서  하락세 를 이어가며  하락폭이  확대됨  \n∙ 아파트  선호 현상과  수도권  아

In [32]:
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)
documents = text_splitter.split_documents(documents)

In [33]:
len(documents)

10

In [34]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.with_openai()

testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

embedding nodes:   0%|          | 0/54 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]